# ClockSandbox — Phase I Smoke Test

- Ideal vs Noisy (white fractional-frequency) over 1 day.
- Deterministic seed.
- Plots of readings and difference, plus basic metrics.

In [ ]:
from src.clocks.ideal import IdealClock
from src.clocks.noisy import NoisyOscillatorClock
from src.analysis import run_clocks, compare_clocks, plot_comparison
ideal = IdealClock()
noisy = NoisyOscillatorClock(sigma_y=1e-11, seed=42)
ts = run_clocks([ideal, noisy], duration=86_400.0, dt=1.0)
metrics = compare_clocks(ts["time"], ts["clock_0"], ts["clock_1"])
metrics

In [ ]:
# Plot (disable if running headless)
plot_comparison(ts, labels=["Ideal", "Noisy (white freq)"])


### Overlapping Allan Deviation with Uncertainties (allantools)

This cell uses **allantools** to compute the **overlapping Allan deviation** \(\sigma_y(	au)\)  
including **1-σ uncertainties**, which are suitable for practical stability analysis.

- Function: `adev_overlapping_allantools(y, dt, taus=None)`
- Plot helper: `plot_adev_with_uncertainties(taus, adev, adev_err)`
- Data type: fractional frequency (`y_k`) sampled every `dt` seconds


In [ ]:
from src.analysis import adev_overlapping_allantools, plot_adev_with_uncertainties
# Reuse 'y' and 'dt' if present; otherwise construct them quickly
try:
    y
    dt
except NameError:
    from src.clocks.ideal import IdealClock
    from src.clocks.noisy import NoisyOscillatorClock
    from src.analysis import run_clocks, fractional_frequency_from_time
    ideal = IdealClock(); noisy = NoisyOscillatorClock(sigma_y=1e-11, seed=7)
    ts = run_clocks([ideal, noisy], duration=12*3600.0, dt=1.0)
    t = ts["time"]; x_noisy = ts["clock_1"]; dt = float(t[1]-t[0])
    y = fractional_frequency_from_time(t, x_noisy, dt=dt)

taus_s, adev, adev_err = adev_overlapping_allantools(y, dt=dt, taus=[1,2,5,10,20,50,100,200,500,1000])
list(zip(taus_s.tolist(), adev.tolist(), adev_err.tolist()))[:5]


In [ ]:
plot_adev_with_uncertainties(taus_s, adev, adev_err)
